In [16]:
path_to_judgment = ['data/gpt3.5-pool-2.txt', 'data/gpt3.5-pool.txt']
path_qrel = ['data/human_qrels']


In [17]:
from collections import defaultdict

def get_pool(paths):
    pool_1 = defaultdict(int)
    for path_1 in paths:
        with open(path_1, 'r') as f:
            lines = f.readlines()

        for line in lines:

            turn_id, _, passage_id, score = line.split(' ')  
            id = turn_id+'****'+ passage_id
            pool_1[id] = int(score.strip())
    return pool_1

gpt_pool = get_pool(path_to_judgment)
nist_pool = get_pool(path_qrel)

In [4]:
def export_holes(judg_main, judg_pred):
    holes = defaultdict(list)
    for id in judg_main:
        if not (id in judg_pred):
            turn_id, pass_id = id.split('****')
            holes[turn_id].append(pass_id)

    return holes

holes = export_holes(nist_pool, gpt_pool)

In [11]:
rows = []
for turn_id in holes:
    for doc_id in holes[turn_id]:
        row = turn_id + '\t0\t' + doc_id + '\n'
        rows.append(row)


with open('docs_to_judge.txt', 'w') as f:
    f.writelines(rows)

In [23]:

pool_1 = defaultdict(list)
path_1 = 'data/human_qrels'

with open(path_1, 'r') as f:
    lines = f.readlines()

for line in lines:
    turn_id, _, passage_id, score = line.split(' ') 
    pool_1[turn_id].append(passage_id) 


In [28]:
path_to_judgment = ['data/gpt3.5-pool-2.txt', 'data/gpt3.5-pool.txt']
judgs = defaultdict(lambda: defaultdict(str))

for path in path_to_judgment:
    with open(path, 'r') as f:
        data = f.readlines()
        for line in data:
            turn_id, _ , doc_id, score = line.split(' ')
            judgs[turn_id][doc_id] = score.strip()

In [29]:
with open('outputs/gpt3.5-V1-pool.txt', 'r') as f:
    data = f.readlines()
    for line in data:
        turn_id, _ , doc_id, score, _ = line.split('\t')
        judgs[turn_id][doc_id] = score

In [32]:
lines = []

for turn_id in judgs:
    for doc_id in judgs[turn_id]:
        if doc_id in pool_1[turn_id]:
            line = turn_id + '\t0\t' + doc_id + '\t' + judgs[turn_id][doc_id] + '\n'
            lines.append(line)

with open('gpt3.5-pool-one-shot.txt', 'w') as f:
    f.writelines(lines)